In [207]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [208]:
PLR_gdf = gpd.read_file('../data/input/berlin/lor_planungsraeume_2021.geojson') # import Planungsräume (PLR) geo data
BZR_gdf = gpd.read_file('../data/input/berlin/lor_bezirksregionen_2021.geojson') # import Bezirksregionen (BZR) geo data
PGR_gdf = gpd.read_file('../data/input/berlin/lor_prognoseraeume_2021.geojson') # import Prognoseräume (PGR) geo data
STREETS_gdf = gpd.read_file('../data/input/berlin/Detailnetz-Strassenabschnitte.geojson') # import Straßen geo data

In [209]:
PLR_gdf.head(5)

,PLR_ID,PLR_NAME,BEZ,STAND,GROESSE_M2,geometry
0,11501341,Karlshorst Süd,11,01.01.2021,2294021.723,"MULTIPOLYGON (((400826.929 5814871.126, 400825..."
1,07601340,Tirschenreuther Ring Ost,07,01.01.2021,413969.326,"MULTIPOLYGON (((387795.701 5807587.153, 387789..."
2,02500831,Wismarplatz,02,01.01.2021,248988.291,"MULTIPOLYGON (((396010.552 5819339.570, 396021..."
3,12601134,Märkisches Zentrum,12,01.01.2021,1127855.841,"MULTIPOLYGON (((387367.428 5828619.763, 387315..."
4,07601547,Horstwalder Straße,07,01.01.2021,941433.954,"MULTIPOLYGON (((391807.174 5805331.900, 391808..."


In [210]:
accident_df = pd.read_csv('../data/temp/raw_accident_dataset_2018-2021.csv')

C:\Users\Domen\AppData\Local\Temp\ipykernel_21420\767954756.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_df = pd.read_csv('../data/temp/raw_accident_dataset_2018-2021.csv')


In [211]:
# accident_df.head(5).T

In [212]:
accident_df = accident_df.drop(columns=["lor", "linrefx", "linrefy"])

In [213]:
accident_gdf = gpd.GeoDataFrame(accident_df, geometry=gpd.points_from_xy(accident_df.longitude, accident_df.latitude),
    crs="EPSG:4326") # transform to GeoDataFrame

### Add PLR Area

In [214]:
accident_gdf = accident_gdf.to_crs(PLR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, PLR_gdf, op='within', how='left') # join

C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [215]:
accident_df['PLR_ID'] = joined_gdf['PLR_ID'] # add to main df
accident_df['PLR_NAME'] = joined_gdf['PLR_NAME'] # add to main df


### Add BZR area

In [216]:
accident_gdf = accident_gdf.to_crs(BZR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, BZR_gdf, predicate='within', how='left') # join

In [217]:
accident_df['BZR_ID'] = joined_gdf['BZR_ID'] # add to main df
accident_df['BZR_NAME'] = joined_gdf['BZR_NAME'] # add to main df


#### ADD PGR area

In [225]:
accident_gdf = accident_gdf.to_crs(PGR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, PGR_gdf, predicate='within', how='left') # join

In [226]:
accident_df['PGR_ID'] = joined_gdf['PGR_ID'] # add to main df
accident_df['PGR_NAME'] = joined_gdf['PGR_NAME'] # add to main df


In [218]:
joined_gdf

,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,ustrzustand,longitude,latitude,geometry,index_right,BZR_ID,BZR_NAME,BEZ,STAND,GROESSE_m2
0,112695,2,Samariterviertel,2500729,2018,1,15,4,3,6,...,1,13.475018,52.513597,POINT (396514.624 5819257.316),15.0,025007,Frankfurter Allee Nord,02,01.01.2021,1.280543e+06
1,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,0,13.291022,52.587259,POINT (384223.627 5827729.725),83.0,125008,West 3 - Borsigwalde/Freie Scholle,12,01.01.2021,4.077143e+06
2,112726,2,Barnimkiez,2400520,2018,1,9,3,3,6,...,0,13.420578,52.526019,POINT (392850.817 5820718.312),120.0,024005,Karl-Marx-Allee Nord,02,01.01.2021,2.227557e+06
3,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,0,13.348288,52.481844,POINT (387834.522 5815915.019),45.0,072003,Schöneberg Südwest,07,01.01.2021,1.150672e+06
4,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,6,...,1,13.403228,52.583472,POINT (391815.267 5827134.254),96.0,032002,Blankenfelde/Niederschönhausen,03,01.01.2021,1.966509e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174,4,0,4300414,2021,3,16,4,3,6,...,1,13.296345,52.511008,POINT (384383.944 5819240.530),88.0,043004,Schloss Charlottenburg,04,01.01.2021,3.516099e+06
50115,111541,4,0,4400726,2021,1,1,6,3,9,...,0,13.289496,52.493813,POINT (383873.882 5817339.107),21.0,044007,Grunewald,04,01.01.2021,7.585200e+06
50116,110370,3,0,2400520,2021,1,20,1,3,3,...,1,13.423095,52.527534,POINT (393025.206 5820883.134),120.0,024005,Karl-Marx-Allee Nord,02,01.01.2021,2.227557e+06
50117,109998,7,0,7300619,2021,1,14,7,3,2,...,1,13.359259,52.474367,POINT (388560.601 5815066.337),121.0,073006,Friedenau Ost,07,01.01.2021,3.141431e+06


In [227]:
accident_df

,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,istsonstige,ustrzustand,longitude,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME
0,112695,2,Samariterviertel,2500729,2018,1,15,4,3,6,...,0,1,13.475018,52.513597,02500729,Pettenkofer Straße,025007,Frankfurter Allee Nord,0250,Friedrichshain Ost
1,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,0,0,13.291022,52.587259,12500824,Ziekowstraße/Freie Scholle,125008,West 3 - Borsigwalde/Freie Scholle,1250,Wittenau/Borsigwalde
2,112726,2,Barnimkiez,2400520,2018,1,9,3,3,6,...,0,0,13.420578,52.526019,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West
3,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,0,0,13.348288,52.481844,07200308,Volkspark (Rudolph-Wilde-Park),072003,Schöneberg Südwest,0720,Schöneberg Süd
4,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,6,...,0,1,13.403228,52.583472,03200206,Pastor-Niemöller-Platz,032002,Blankenfelde/Niederschönhausen,0320,Nördliches Pankow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174,4,0,4300414,2021,3,16,4,3,6,...,0,1,13.296345,52.511008,04300414,Schloßstraße,043004,Schloss Charlottenburg,0430,Charlottenburg Zentrum
50115,111541,4,0,4400726,2021,1,1,6,3,9,...,0,0,13.289496,52.493813,04400726,Bismarckallee,044007,Grunewald,0440,Wilmersdorf Süd
50116,110370,3,0,2400520,2021,1,20,1,3,3,...,0,1,13.423095,52.527534,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West
50117,109998,7,0,7300619,2021,1,14,7,3,2,...,0,1,13.359259,52.474367,07300619,Grazer Platz,073006,Friedenau Ost,0730,Friedenau
